<a href="https://colab.research.google.com/github/realpranav93/EVA5/blob/master/S4/EVA5_Session_4_pranav.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [19]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        #block1
        self.conv1 = nn.Conv2d(1, 10, 3) #26
        self.Batchnorm1 = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 18, 3) #24
        self.Batchnorm2 = nn.BatchNorm2d(18)
        self.conv3 = nn.Conv2d(18,18,3) #22
        self.Batchnorm3 = nn.BatchNorm2d(18)
        self.dp3 = nn.Dropout(p = 0.20)

        #transition block
        self.pool1 = nn.MaxPool2d(2, 2) #11
        self.pool1trns = nn.Conv2d(18, 10, 1)#11
        self.Batchnormtrns1 = nn.BatchNorm2d(10)
        
        #block2
        self.conv4 = nn.Conv2d(10, 18, 3)#9
        self.Batchnorm4 = nn.BatchNorm2d(18)
        self.dp4 = nn.Dropout(p = 0.20)
        self.conv5 = nn.Conv2d(18, 26, 3)#7
        self.Batchnorm5 = nn.BatchNorm2d(26)
        self.dp5 = nn.Dropout(p = 0.20)

        #block3
        self.pool2trns = nn.Conv2d(26,10,1) #7
        self.Batchnormtrns2 = nn.BatchNorm2d(10)
        self.conv6 = nn.Conv2d(10,10,7) #1


    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.Batchnorm1(x)
        
        #x = self.dropout(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.Batchnorm2(x)

        
        #x = self.dropout(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.Batchnorm3(x)
        x = self.dp3(x)
        
        #x = self.dropout(x)
        x = self.pool1(x)
        x = self.pool1trns(x)
        x = F.relu(x)
        x = self.Batchnormtrns1(x)
        
        x = self.conv4(x)
        x = F.relu(x)
        x = self.Batchnorm4(x)
        x = self.dp4(x)


        x = self.conv5(x)
        x = F.relu(x)
        x = self.Batchnorm5(x)
        x = self.dp5(x)
        
        x = self.pool2trns(x)
        x = F.relu(x)
        x = self.Batchnormtrns2(x)


        x = self.conv6(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

In [20]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
       BatchNorm2d-2           [-1, 10, 26, 26]              20
            Conv2d-3           [-1, 18, 24, 24]           1,638
       BatchNorm2d-4           [-1, 18, 24, 24]              36
            Conv2d-5           [-1, 18, 22, 22]           2,934
       BatchNorm2d-6           [-1, 18, 22, 22]              36
           Dropout-7           [-1, 18, 22, 22]               0
         MaxPool2d-8           [-1, 18, 11, 11]               0
            Conv2d-9           [-1, 10, 11, 11]             190
      BatchNorm2d-10           [-1, 10, 11, 11]              20
           Conv2d-11             [-1, 18, 9, 9]           1,638
      BatchNorm2d-12             [-1, 18, 9, 9]              36
          Dropout-13             [-1, 18, 9, 9]               0
           Conv2d-14             [-1, 2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:75: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [21]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [15]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    #pbar = tqdm(train_loader)
    train_loss = 0 
    train_correct = 0
    #scheduler.step()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        #pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        train_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        train_correct += pred.eq(target.view_as(pred)).sum().item()

    train_loss /= len(test_loader.dataset)
    print('Epoch: {:.0f},LR: {}.\nTrain set: train Average loss: {:.4f}, train_Accuracy: {}/{} ({:.4f}%)\n'.format(
        epoch,optimizer.param_groups[0]['lr'],train_loss, train_correct, len(train_loader.dataset),
        100. * train_correct / len(train_loader.dataset)))
        

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred_test = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred_test.eq(target.view_as(pred_test)).sum().item()

    test_loss /= len(test_loader.dataset)

    
    print('Test set: test Average loss: {:.4f}, test Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [27]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:75: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 1,LR: 0.01.
Train set: train Average loss: 0.0871, train_Accuracy: 59725/60000 (99.5417%)

Test set: test Average loss: 0.0214, test Accuracy: 9944/10000 (99.4400%)

Epoch: 2,LR: 0.01.
Train set: train Average loss: 0.0906, train_Accuracy: 59707/60000 (99.5117%)

Test set: test Average loss: 0.0210, test Accuracy: 9932/10000 (99.3200%)

Epoch: 3,LR: 0.01.
Train set: train Average loss: 0.0849, train_Accuracy: 59696/60000 (99.4933%)

Test set: test Average loss: 0.0218, test Accuracy: 9929/10000 (99.2900%)

Epoch: 4,LR: 0.01.
Train set: train Average loss: 0.0840, train_Accuracy: 59731/60000 (99.5517%)

Test set: test Average loss: 0.0260, test Accuracy: 9922/10000 (99.2200%)

Epoch: 5,LR: 0.01.
Train set: train Average loss: 0.0776, train_Accuracy: 59752/60000 (99.5867%)

Test set: test Average loss: 0.0245, test Accuracy: 9920/10000 (99.2000%)

Epoch: 6,LR: 0.01.
Train set: train Average loss: 0.0804, train_Accuracy: 59727/60000 (99.5450%)

Test set: test Average loss: 0.0211, 